# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.3 kB ? eta -:--:--
     ---------- --------------------------- 276.5/990.3 kB 5.7 MB/s eta 0:00:01
     ------------------------ ------------- 645.1/990.3 kB 6.8 MB/s eta 0:00:01
     -------------------------- ----------- 696.3/990.3 kB 7.3 MB/s eta 0:00:01
     -------------------------------------- 990.3/990.3 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/374.2 kB ? eta -:--:--
     ------------------------------------- 374.2/374.2 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.22
    Uninstalling langchain-core-0.2.22:
      Successfully uninstalled langchain-core-0.2.22
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.10
    Uninstalling langchain-0.2.10:
      Successfully uninstalled langchain-0.2.10


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物学家。你擅长回答生物相关问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机专家。你擅长回答计算机相关问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_language_and_literature_template = """你是一位很棒的汉语言文学家。你擅长回答汉语言问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_language_and_literature_template,
    },
]

In [17]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url='https://api.xiaoai.plus/v1',api_key='sk-38rkK0xdFDDMClm688CbCb0204F64d3d874066CbEf2fE608')

In [18]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [19]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [20]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [21]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [22]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [23]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [24]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [25]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [26]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [27]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射？'}
> Finished chain.
{'input': '什么是黑体辐射？', 'text': '\n\n黑体辐射是一种理想化的热辐射体，它能够吸收所有入射的电磁辐射，无论波长如何，也能够放射出同样频率和强度的辐射。它的特点是能够达到热平衡状态，即吸收的能量等于放射出的能量，而不会有净的能量增加或减少。黑体辐射是研究热辐射和热力学的重要基础。'}


In [28]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number larger than 40 that can be divided by 3?'}
> Finished chain.
{'input': 'What is the first prime number larger than 40 that can be divided by 3?', 'text': '\n\nThe first prime number larger than 40 that can be divided by 3 is 43.'}


In [29]:
print(
    chain.invoke(
        "血液是由什么组成？"
    )
)



> Entering new MultiPromptChain chain...
生物: {'input': '什么是血液？'}
> Finished chain.
{'input': '什么是血液？', 'text': '\n\n血液是由血浆和细胞组成的复杂液体。血浆主要由水、蛋白质和其他营养物质组成，它在输送营养物质、荷尔蒙、药物和废物的同时，还起着调节体温和维持水平衡的作用。血液细胞包括红细胞、白细胞和血小板，它们在体内提供氧气、免疫保护和止血功能。血液也承担着运输和保护身体免受感染的重要作用，是维持人体生命活动的必需品。'}


In [30]:
print(
    chain.invoke(
        "java的数据类型有哪些？"
    )
)



> Entering new MultiPromptChain chain...
计算机: {'input': 'java的数据类型有哪些？'}
> Finished chain.
{'input': 'java的数据类型有哪些？', 'text': '\n\nJava的数据类型包括基本数据类型和引用数据类型。基本数据类型包括：byte、short、int、long、float、double、char和boolean。引用数据类型包括：类、接口、数组等。'}


In [31]:
print(
    chain.invoke(
        "四大名著是什么？还有他们的作者是谁？"
    )
)



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '四大名著是什么？还有他们的作者是谁？'}
> Finished chain.
{'input': '四大名著是什么？还有他们的作者是谁？', 'text': '\n\n四大名著是指中国古典文学中最具代表性的四部作品，分别是《红楼梦》、《三国演义》、《水浒传》和《西游记》。\n《红楼梦》的作者是曹雪芹，许多人认为另一位作者是高鹗；《三国演义》的作者是罗贯中；《水浒传》的作者是施耐庵；《西游记》的作者是吴承恩。'}


In [32]:
router_chain.verbose = True

In [33]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是宇宙中非常密集的天体，它具有非常强大的引力，甚至连光也无法逃逸。这种强大的引力是由于黑洞内部的物质密度非常高，导致其产生巨大的引力场。黑洞是由恒星坍缩而成的，当恒星燃尽所有的燃料后，它会坍缩成一个极小的点，这就是黑洞。黑洞在宇宙中起着重要的作用，它们可以吸收周围的物质，甚至整个星系。但是关于黑洞的许多奥秘仍然未解开，我们仍在努力研究和探索它们。'}


In [34]:
print(
    chain.invoke(
        "质数包括1吗？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
数学: {'input': '质数包括1吗？'}
> Finished chain.
{'input': '质数包括1吗？', 'text': '\n\n不包括。质数是指只能被1和自身整除的正整数，而1既不是质数也不是合数，因为它只能被自身整除。因此，质数不包括1。'}


In [36]:
print(
    chain.invoke(
        "为什么1不是质数？质数是指只能被1和自身整除的正整数，1能被1整除，也能被自身整除的正整数呀"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
数学: {'input': '为什么1不是质数？质数是指只能被1和自身整除的正整数，1能被1整除，也能被自身整除的正整数呀'}
> Finished chain.
{'input': '为什么1不是质数？质数是指只能被1和自身整除的正整数，1能被1整除，也能被自身整除的正整数呀', 'text': '。\n\n这是因为质数的定义是只能被1和自身整除，而1只能被1整除，不能被自身整除，因此不符合质数的定义。'}


In [37]:
print(
    chain.invoke(
        "介绍一下减数分裂"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '介绍一下减数分裂'}
> Finished chain.
{'input': '介绍一下减数分裂', 'text': '\n\n减数分裂是生物细胞分裂的一种形式，也称为减数减分裂。它发生在生殖细胞（例如精子和卵子）中，用于生殖和遗传的目的。减数分裂是一种有性生殖过程，它通过将染色体数量减半来产生具有不同基因组的子代。这是通过两次细胞分裂来完成的，包括一次减数分裂（第一次分裂）和一次等于分裂（第二次分裂）。在减数分裂的第一次分裂中，染色体从配子细胞中分开，每个配子细胞只有一套染色体。在第二次分裂中，每个配子细胞再次分裂，产生四个不同的细胞，每个细'}


In [38]:
print(
    chain.invoke(
        "正则表达式是怎么实现的？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'How is regular expression implemented?'}
> Finished chain.
{'input': 'How is regular expression implemented?', 'text': '\n\n正则表达式是通过使用特定的语法和符号来定义匹配模式，然后通过一个算法来检查给定的字符串是否符合这个模式。这个算法可以是基于有限状态机、递归下降解析器或者其他方法。在实际实现中，通常会使用编程语言中的字符串处理函数来实现正则表达式的功能。'}


In [39]:
print(
    chain.invoke(
        "周瑜是被诸葛亮气死的吗？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '周瑜是被诸葛亮气死的吗？'}
> Finished chain.
{'input': '周瑜是被诸葛亮气死的吗？', 'history': '', 'text': ' 据历史记录显示，周瑜并非被诸葛亮气死。事实上，周瑜是死于病逝，而不是因为任何气恼事件。据说，周瑜在死前还写信给诸葛亮，并称赞他为“卧龙”。'}


In [40]:
print(
    chain.invoke(
        "周瑜是被诸葛亮气死的吗？这个问题不是汉语言文学相关吗？"
    )
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '周瑜是被诸葛亮气死的吗？这个问题不是汉语言文学相关吗？'}
> Finished chain.
{'input': '周瑜是被诸葛亮气死的吗？这个问题不是汉语言文学相关吗？', 'text': '\n\n虽然这个问题涉及到汉语文学中的人物和故事，但是它更多地涉及到历史和传说的问题。所以，作为汉语言文学家，我并不具备回答这个问题的专业知识，建议你去寻找历史学或传说研究领域的专家来解答。同时，我们也要注意区分历史和文学作品中的人物和故事，避免混淆。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains